# Group Project A Report

## Can We Predict the Presence of Heart Diseases from Observing Resting Blood Pressure and Serum Cholesterol? 

### Introduction:







### Exploratory Work on the Data

In [39]:
library(tidyverse)
library(tidymodels)
library(repr)
library(RColorBrewer)

In [25]:
heart_data <- read_csv("https://raw.githubusercontent.com/oskarjychen/dsci100_group_a/main/data/processed.cleveland.data",
                       col_names = FALSE)

selected_heart_data <- heart_data |>
    select(trestbps = X4, chol = X5, num = X14) 
selected_heart_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


trestbps,chol,num
<dbl>,<dbl>,<dbl>
145,233,0
160,286,2
120,229,1
130,250,0
130,204,0
120,236,0
140,268,3
120,354,0
130,254,2


*figure 1*

Since the *num* variable is in essence categorical, we will creat a new binary column to signify whether heart disease in present in the patient. The new column will be called heart_disease, with the *num* value of 0 represented by "none", and the values of 1, 2, 3, 4 being represented by "present".

We will also be changing both *num* and *heart_disease* to factor columns:

In [35]:
clean_heart_data <- selected_heart_data |>
    mutate(heart_disease = ifelse(num > 0, "present", "none")) |>
    mutate(num = as_factor(num), heart_disease = as_factor(heart_disease))
clean_heart_data

trestbps,chol,num,heart_disease
<dbl>,<dbl>,<fct>,<fct>
145,233,0,none
160,286,2,present
120,229,1,present
130,250,0,none
130,204,0,none
120,236,0,none
140,268,3,present
120,354,0,none
130,254,2,present


*figure 2*

Now we can look at how the presence of heart diseases in this data frame is distributed:

In [37]:
num_obs <- nrow(clean_heart_data)
clean_heart_data |>
    group_by(heart_disease) |>
    summarize(count = n(), percentage = n()/num_obs * 100)

heart_disease,count,percentage
<fct>,<int>,<dbl>
none,164,54.12541
present,139,45.87459


*figure 3*

We can see that patients with no heart disease account for 54.13%, while those with heart disease make up for around 45.87%